In [3]:
from config.config import Config
from parsing.directory import list_files_in_directory
from parsing.parse_xls import XLSParser, RawWood
from taxation.volumes_table import Volume
from wood.wood import Wood, Trunk

config = Config()

def parse() -> list[Wood]:

    # Чтение файлов таблиц
    files_for_parse = list_files_in_directory(config.settings.parse_directory)
    raw_woods = []
    for file in files_for_parse:
        wood_of_file = XLSParser().parse(file)
        raw_woods.extend(wood_of_file)

    woods = []
    # Парсинг таблиц
    for wood_list in raw_woods:
        raw_wood = RawWood(*wood_list)
        try:
            if raw_wood.is_valid():
                for wood in raw_wood.parse():
                    woods.append(wood)
        except Exception as e:
            print(e)

    # TODO: Обработка непрочитанных деревьев

    for wood in woods:
        print(wood)

    return woods

source_woods = parse()
for wood in source_woods:
    print(wood)

In [40]:
import json
from typing import Literal

class UnknownSpecie(Exception):
    pass

class ParseDensityError(Exception):
    pass

class Density:
    def __init__(self):
        try:
            with open('config/density.json', 'r', encoding='utf-8') as file:
                data = json.load(file)
                self.group_of_wood = data.get('group_of_wood', {})
                self.density_dict = data.get('density_dict', {})
        except Exception as e:
            print("Ошибка чтения файла density.json", e)

    def get_density(self, specie: str) -> int:
        """
        Определяет плотность породы.
        :param specie: порода
        :return: плотность кг/м3
        """
        for group, species_list in self.group_of_wood.items():
            if specie in species_list:
                return self.density_dict.get(group)
        raise UnknownSpecie(f"Попытка получения плотности для неизвестной породы '{specie}'.")

    def add_specie_to_group(self, group: Literal["мягкие", "твердые", "лиственница"], specie: str) -> None:
        """
        Добавляет новый вид древесины в указанный список группы.
        :param group: группа плотности ("мягкие", "твердые" или "лиственница")
        :param specie: порода древесины
        :return: None
        """

        if group not in self.group_of_wood:
            raise ValueError(f"Группа '{group}' не найдена. Доступные группы: "
                             f"{', '.join(self.group_of_wood.keys())}")

        if specie in self.group_of_wood[group]:
            print(f"Порода '{specie}' уже присутствует в группе '{group}'.")
            return

        self.group_of_wood[group].append(specie)
        try:
            with open('config/density.json', 'w', encoding='utf-8') as file:
                json.dump({
                    'group_of_wood': self.group_of_wood,
                    'density_dict': self.density_dict
                }, file, ensure_ascii=False, indent=4)
        except Exception as e:
            print("Ошибка записи в файл density.json", e)


In [59]:
import openpyxl

class WoodWaste(Wood):
    """
    Класс для расчёта отходов.
    """
    density: float
    data: list
    def __init__(self, name: str, number: str, specie: str, is_shrub: bool, trunks: list[Trunk],
                 area: float = None):
        super().__init__(name, number, specie, is_shrub, trunks, area)

        self.calculation()

    def calculation(self) -> None:
        # нахождение объёмов
        volume = Volume()
        for trunk in self.trunks:
            trunk.volume = volume.get_volume(trunk.diameter, trunk.height)
        # перемещение ствола с наибольшим объёмом в начало списка
        max_trunk = max(self.trunks, key=lambda t: t.volume)
        self.trunks.remove(max_trunk)
        self.trunks.insert(0, max_trunk)
        # определение плотности породы
        try:
            self.density = Density().get_density(self.specie)
        except UnknownSpecie as e:
            print(e)
            group = input(f"К какой группе плотности относится пород '{self.specie}'?\n"
                          f"1-мягкие, 2-твердые, 3-лиственница. Введите цифру: ")   # TODO
            group = {1:"мягкие", 2:"твердые", 3:"лиственница"}[group]
            Density().add_specie_to_group(group, self.specie)
            self.density = Density().get_density(self.specie)

    def export_preparation(self) -> None:
        self.data = []
        if not self.is_shrub:
            for idx, trunk in enumerate(self.trunks):
                self.data.append(
                    [self.number if idx == 0 else None, self.name if idx == 0 else None,
                     1, trunk.diameter, trunk.height, trunk.volume, self.density]
                )
        else:
            self.data.append(
                [self.number, self.name, 3 if self.area else 1, self.trunks[0].diameter, self.trunks[0].height,
                 self.trunks[0].volume, self.area if self.area else 1/3, self.density]
                )

    @staticmethod
    def export_to_xls(data, path) -> None:
        # Создаем новую рабочую книгу и выбираем активный лист
        workbook = openpyxl.Workbook()
        sheet = workbook.active

        # Переменная для отслеживания текущей строки в Excel
        current_row = 1

        # Проходим по внешнему списку
        for group in data:
            for row in group:
                # Записываем данные в строки листа Excel
                sheet.append(row)
                current_row += 1

        # Сохраняем рабочую книгу в файл
        workbook.save(path)
        print(f"Успешно {path}.")

    def __repr__(self) -> str:
        if not self.is_shrub:
            s = f"{self.number} | {self.name} | 1 | {self.trunks[0].diameter} | {self.trunks[0].height} | " \
                f"{self.trunks[0].volume} | {self.density}"
            s_ = ""
            for trunk in self.trunks[1:]:
                s_ += f"\n\t|\t| 1 | {trunk.diameter} | {trunk.height} | {trunk.volume} | {self.density}"
            return s+s_
        else:
            s = f"{self.number} | {self.name} | {3 if self.area else 1} | {self.trunks[0].diameter} | " \
                f"{self.trunks[0].height} | {self.trunks[0].volume} | {self.area if self.area else 1/3} | " \
                f"{self.density}"
            return s


In [55]:
for wood in source_woods:
    print(WoodWaste(wood.name, wood.number, wood.specie, wood.is_shrub, wood.trunks, wood.area))

1 | Береза пушистая | 1 | 4.0 | 5.0 | 0.0063 | 870
2 | Ель обыкновенная | 1 | 4.0 | 5.0 | 0.0063 | 762
3 | Ель обыкновенная | 1 | 4.0 | 5.0 | 0.0063 | 762
4 | Ель обыкновенная | 1 | 4.0 | 5.0 | 0.0063 | 762
5 | Ель обыкновенная | 1 | 4.0 | 5.0 | 0.0063 | 762
6 | Ольха серая | 1 | 4.0 | 5.0 | 0.0063 | 762
7 | Ольха серая | 1 | 5.0 | 5.0 | 0.0098 | 762
8 | Ель обыкновенная | 1 | 4.0 | 5.0 | 0.0063 | 762
9 | Ель обыкновенная | 1 | 4.0 | 4.0 | 0.005 | 762
10 | Ольха серая | 1 | 4.0 | 5.0 | 0.0063 | 762
11 | Ольха серая | 1 | 4.0 | 5.0 | 0.0063 | 762
12 | Ольха серая | 1 | 4.0 | 5.0 | 0.0063 | 762
13 | Ольха серая | 1 | 4.0 | 5.0 | 0.0063 | 762
14 | Ольха серая | 1 | 4.0 | 5.0 | 0.0063 | 762
15 | Ольха серая | 1 | 4.0 | 5.0 | 0.0063 | 762
16 | Ива ломкая (54 ствола) | 1 | 7.0 | 5.0 | 0.0192 | 762
	|	| 1 | 7.0 | 5.0 | 0.0192 | 762
	|	| 1 | 7.0 | 5.0 | 0.0192 | 762
	|	| 1 | 4.0 | 5.0 | 0.0063 | 762
	|	| 1 | 4.0 | 5.0 | 0.0063 | 762
	|	| 1 | 4.0 | 5.0 | 0.0063 | 762
	|	| 1 | 4.0 | 5.0 | 0.0063

In [64]:
result_wood = []
result_shrub = []
for wood in source_woods:
    w = WoodWaste(wood.name, wood.number, wood.specie, wood.is_shrub, wood.trunks, wood.area)
    w.export_preparation()
    if w.is_shrub:
        result_shrub.append(w.data)
    else:
        result_wood.append(w.data)
from pprint import pprint
pprint(result_wood)
pprint(result_shrub)
WoodWaste.export_to_xls(result_wood, "data/out/wood_list_example_1_out_wood.xlsx")
WoodWaste.export_to_xls(result_shrub, "data/out/wood_list_example_1_out_shrub.xlsx")

[[['1', 'Береза пушистая', 1, 4.0, 5.0, 0.0063, 870]],
 [['2', 'Ель обыкновенная', 1, 4.0, 5.0, 0.0063, 762]],
 [['3', 'Ель обыкновенная', 1, 4.0, 5.0, 0.0063, 762]],
 [['4', 'Ель обыкновенная', 1, 4.0, 5.0, 0.0063, 762]],
 [['5', 'Ель обыкновенная', 1, 4.0, 5.0, 0.0063, 762]],
 [['6', 'Ольха серая', 1, 4.0, 5.0, 0.0063, 762]],
 [['7', 'Ольха серая', 1, 5.0, 5.0, 0.0098, 762]],
 [['8', 'Ель обыкновенная', 1, 4.0, 5.0, 0.0063, 762]],
 [['9', 'Ель обыкновенная', 1, 4.0, 4.0, 0.005, 762]],
 [['10', 'Ольха серая', 1, 4.0, 5.0, 0.0063, 762]],
 [['11', 'Ольха серая', 1, 4.0, 5.0, 0.0063, 762]],
 [['12', 'Ольха серая', 1, 4.0, 5.0, 0.0063, 762]],
 [['13', 'Ольха серая', 1, 4.0, 5.0, 0.0063, 762]],
 [['14', 'Ольха серая', 1, 4.0, 5.0, 0.0063, 762]],
 [['15', 'Ольха серая', 1, 4.0, 5.0, 0.0063, 762]],
 [['16', 'Ива ломкая (54 ствола)', 1, 7.0, 5.0, 0.0192, 762],
  [None, None, 1, 7.0, 5.0, 0.0192, 762],
  [None, None, 1, 7.0, 5.0, 0.0192, 762],
  [None, None, 1, 4.0, 5.0, 0.0063, 762],
  [None,